# High-level Gluon Example

In [1]:
import os
import sys
import numpy as np
import math
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import json
from common.params import *
from common.utils import *

In [2]:
ctx = mx.gpu()

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("MXNet: ", mx.__version__)
print("Numpy: ", np.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]
MXNet:  1.0.0
Numpy:  1.14.0
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']


In [4]:
def SymbolModule():
    sym = gluon.nn.Sequential()
    with sym.name_scope():
        sym.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1, activation='relu'))
        sym.add(gluon.nn.Conv2D(channels=50, kernel_size=3, padding=1))
        sym.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        sym.add(gluon.nn.Activation('relu'))
        # Equiv to gluon.nn.LeakyReLU(0)
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1, activation='relu'))
        sym.add(gluon.nn.Conv2D(channels=100, kernel_size=3, padding=1))
        sym.add(gluon.nn.MaxPool2D(pool_size=2, strides=2))
        sym.add(gluon.nn.Activation('relu'))
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Flatten())
        sym.add(gluon.nn.Dense(512, activation='relu'))
        sym.add(gluon.nn.Dropout(0.25))
        sym.add(gluon.nn.Dense(N_CLASSES))
    return sym

In [5]:
def init_model(m):
    trainer = gluon.Trainer(m.collect_params(), 'sgd',
                            {'learning_rate': LR, 'momentum':MOMENTUM})
    criterion = gluon.loss.SoftmaxCrossEntropyLoss()
    return trainer, criterion

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000,) (10000,)
float32 float32 int32 int32
CPU times: user 608 ms, sys: 625 ms, total: 1.23 s
Wall time: 1.23 s


In [7]:
%%time
sym = SymbolModule()
sym.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

CPU times: user 972 ms, sys: 668 ms, total: 1.64 s
Wall time: 1.67 s


In [8]:
%%time
trainer, criterion = init_model(sym)

CPU times: user 196 µs, sys: 102 µs, total: 298 µs
Wall time: 305 µs


In [9]:
results=dict()
with Timer() as t:
    # Sets training = True 
    for j in range(EPOCHS):
        train_loss = 0.0
        for data, target in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
            # Get samples
            data = nd.array(data).as_in_context(ctx)
            target = nd.array(target).as_in_context(ctx)
            with autograd.record():
                # Forwards
                output = sym(data)
                # Loss
                loss = criterion(output, target)
            # Back-prop
            loss.backward()
            trainer.step(data.shape[0])
            train_loss += nd.sum(loss).asscalar()
        # Log
        print('Epoch %3d: loss: %5.4f'%(j, train_loss/len(x_train)))
print('Training took %.03f sec.' % t.interval)
results['training duration']=t.interval

Epoch   0: loss: 1.8273
Epoch   1: loss: 1.3419
Epoch   2: loss: 1.1263
Epoch   3: loss: 0.9500
Epoch   4: loss: 0.8210
Epoch   5: loss: 0.7102
Epoch   6: loss: 0.6173
Epoch   7: loss: 0.5375
Epoch   8: loss: 0.4638
Epoch   9: loss: 0.3909
Training took 61.066 sec.


In [10]:
%%time
# Test model
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, target in yield_mb(x_test, y_test, BATCHSIZE):
    # Get samples
    data = nd.array(data).as_in_context(ctx)
    # Forwards
    output = sym(data)
    pred = nd.argmax(output, axis=1)
    # Collect results
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = pred.asnumpy()
    c += 1

CPU times: user 508 ms, sys: 131 ms, total: 639 ms
Wall time: 420 ms


In [11]:
acc=sum(y_guess == y_truth)/float(len(y_guess))
print("Accuracy: ", acc)
results['accuracy']=t.interval

Accuracy:  0.7593149038461539


In [12]:
with open('gluon_cnn_{}.json'.format(get_gpu_name()[0].split(' ')[1].split('-')[0]), 'w') as f:
    json.dump(results, f)